In [1]:
### Palm2 and Langchain installation
! pip install google-generativeai langchain --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.8 MB/s eta 0:00:00


In [8]:
! pip install -U langchain-community --quiet

In [11]:
!pip install langchain_google_genai --quiet




### **Palm2 setup with Langchain**



In [12]:
from google.colab import userdata
PALM2_API_KEY = userdata.get('PALM2_API_KEY')

In [14]:
from langchain.embeddings import GooglePalmEmbeddings
#from langchain.llms import GooglePalm
from langchain_google_genai import GoogleGenerativeAI

llm = GoogleGenerativeAI(google_api_key=PALM2_API_KEY,
                 temperature=0.1,
                top_k=1,model="gemini-pro")

In [15]:
llm

GoogleGenerativeAI(model='gemini-pro', google_api_key=SecretStr('**********'), temperature=0.1, top_k=1, client=genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
))

In [16]:
### testing the set up
prompts = ["Who won the last football world cup?"] # according to the class prompts must be in list
llm_result = llm._generate(prompts)
llm_result

LLMResult(generations=[[Generation(text='Argentina')]], llm_output=None, run=None)


### **SimpleSequenctialChain**


In [17]:
from langchain import PromptTemplate,  LLMChain # To create PromptTemplate and LLMChain
from langchain.chains import SimpleSequentialChain , SequentialChain


In [18]:
template = 'Which country got independence in {year}'
prompt_1 = PromptTemplate(template=template, input_variables=["year"])
chain_1 = LLMChain(llm=llm,prompt=prompt_1)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [19]:
template = 'Please let me the after independence 5 historical event of that country: {country}'
prompt_2 = PromptTemplate(template=template, input_variables=["country"])
chain_2 = LLMChain(llm=llm,prompt=prompt_2)

In [20]:
complete_chain = SimpleSequentialChain(chains=[chain_1,chain_2],
                                      verbose=True)

In [21]:
output = complete_chain.run('1947')
print(output)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
India
1. **Partition of India (1947):** The division of British India into the independent dominions of India and Pakistan, leading to widespread violence and displacement.

2. **Adoption of the Indian Constitution (1950):** The drafting and adoption of the Indian Constitution, which established India as a sovereign, democratic republic.

3. **Green Revolution (1960s-1970s):** A period of agricultural modernization that led to increased crop yields and reduced food shortages.

4. **Indo-Pakistani War of 1971:** A conflict that resulted in the liberation of Bangladesh from Pakistan and the emergence of India as a regional power.

5. **Economic Reforms of 1991:** A series of economic liberalization measures that opened up the Indian economy to foreign investment and trade, leading to significant economic growth.

> Finished chain.
1. **Partition of India (1947):** The division of British India into the independent dominions of India and Pak

In [22]:
output = complete_chain.run('1971')
print(output)



> Entering new SimpleSequentialChain chain...
Bangladesh
1. **Liberation War (1971):** The Bangladesh Liberation War, fought against Pakistan, resulted in the independence of Bangladesh on December 16, 1971.

2. **Adoption of the Constitution (1972):** The first constitution of Bangladesh was adopted on November 4, 1972, establishing a parliamentary democracy.

3. **Assassination of Sheikh Mujibur Rahman (1975):** The founding father of Bangladesh, Sheikh Mujibur Rahman, was assassinated on August 15, 1975, along with most of his family.

4. **Military Coups and Martial Law (1975-1990):** Bangladesh experienced a series of military coups and martial law regimes following Sheikh Mujibur Rahman's assassination.

5. **Return to Democracy (1991):** In 1991, Bangladesh returned to democracy with the election of Begum Khaleda Zia as Prime Minister.

> Finished chain.
1. **Liberation War (1971):** The Bangladesh Liberation War, fought against Pakistan, resulted in the independence of Bangla

### **Download data**

In [23]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [25]:
! kaggle datasets download -d PromptCloudHQ/amazon-reviews-unlocked-mobile-phones

Dataset URL: https://www.kaggle.com/datasets/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
License(s): CC0-1.0
 77% 25.0M/32.6M [00:00<00:00, 58.0MB/s]
100% 32.6M/32.6M [00:00<00:00, 66.3MB/s]


In [26]:
!unzip amazon-reviews-unlocked-mobile-phones.zip


Archive:  amazon-reviews-unlocked-mobile-phones.zip
  inflating: Amazon_Unlocked_Mobile.csv  


### **SequentialChain**

In [27]:
import pandas as pd
df = pd.read_csv('Amazon_Unlocked_Mobile.csv')


In [28]:
df.dropna(inplace=True)


In [29]:
# Input text

review = df['Reviews'][15]
review

'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.'

In [30]:
templete_1 = "Identify the key issues in the given mobile review: \n{review}"
prompt_1 = PromptTemplate(template=templete_1,
                          input_variables=['review'])
chain_1 = LLMChain(llm=llm,
                  prompt=prompt_1,
                  output_key='issues')

In [31]:
templete_2 = "Write a constructive steps needs to be taken by the manufacturing company \
             based on the issues: \n{issues}"
prompt_2 = PromptTemplate(template=templete_2,
                          input_variables=['issues'])
chain_2 = LLMChain(llm=llm,
                  prompt=prompt_2,
                  output_key='final_actions')

In [32]:
seq_chain = SequentialChain(chains=[chain_1,chain_2],
                            input_variables=['review'],
                            output_variables=['issues','final_actions']
                           )

In [33]:
output = seq_chain(review)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [34]:
print(output)


{'review': 'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.', 'issues': '**Key Issues:**\n\n* SD card slot not opening when accessed in file management\n* SD card not recognized by computer\n* SD card is functional on other devices', 'final_actions': "**Constructive Steps for Manufacturing Company:**\n\n**1. Investigate Hardware Issue:**\n\n* Inspect the SD card slot for any physical damage or obstructions.\n* Test the slot with a known-working SD card to confirm functionality.\n* Check the device's firmware and update if necessary.\n\n**2. Troubleshoot Software Issue:**\n\n* Ensure that the file management application is up-to-date.\n* Check for any corrupted files or software conflicts that may be interfering with SD card access.\n* Reset the 

In [35]:
print(output['issues'])


**Key Issues:**

* SD card slot not opening when accessed in file management
* SD card not recognized by computer
* SD card is functional on other devices


In [36]:
print(output['final_actions'])


**Constructive Steps for Manufacturing Company:**

**1. Investigate Hardware Issue:**

* Inspect the SD card slot for any physical damage or obstructions.
* Test the slot with a known-working SD card to confirm functionality.
* Check the device's firmware and update if necessary.

**2. Troubleshoot Software Issue:**

* Ensure that the file management application is up-to-date.
* Check for any corrupted files or software conflicts that may be interfering with SD card access.
* Reset the device to factory settings to eliminate any software glitches.

**3. Test with Different SD Cards:**

* Use multiple SD cards of different capacities and brands to rule out any compatibility issues.
* Format the SD cards using the device's recommended file system.

**4. Check Computer Compatibility:**

* Verify that the computer's operating system and hardware are compatible with the SD card.
* Try using different USB ports or card readers to eliminate any connection issues.

**5. Contact Customer Suppor

### **TransformChain**

In [37]:
def preprocessing(inputs: dict) -> dict:
    lower = inputs['review'].lower()
    lower_replace = lower.replace("!","").replace("?","")
    return {"output":lower_replace}

In [38]:
review

'Had this phone before and loved it but was not working so I got this phone. One thing is the SD card slot does not open up when I try to access it in file managment, even does not show up on the computer. The card is fine was able to open on another phone. Trying to trouble shoot it for now.'

In [39]:
review_processed = preprocessing({'review': review})
print(review_processed)

{'output': 'had this phone before and loved it but was not working so i got this phone. one thing is the sd card slot does not open up when i try to access it in file managment, even does not show up on the computer. the card is fine was able to open on another phone. trying to trouble shoot it for now.'}


In [40]:
from langchain.chains import TransformChain

tf_chain = TransformChain(input_variables = ['review'],
                                   output_variables = ['output'],
                                   transform = preprocessing)

In [42]:
tf_chain.run({'review': review})


'had this phone before and loved it but was not working so i got this phone. one thing is the sd card slot does not open up when i try to access it in file managment, even does not show up on the computer. the card is fine was able to open on another phone. trying to trouble shoot it for now.'

In [43]:
template_tf = """Please write a one liner summary of the given mobile review:
\n{output}"""

In [45]:
prompt_tf = PromptTemplate(input_variables=['output'],
                           template=template_tf)
summary_chain = LLMChain(llm=llm,
                         prompt=prompt_tf)

In [46]:
seq_chain = SimpleSequentialChain(chains=[tf_chain,summary_chain],
                                  verbose=True)

In [47]:
result = seq_chain.run(review)




> Entering new SimpleSequentialChain chain...
had this phone before and loved it but was not working so i got this phone. one thing is the sd card slot does not open up when i try to access it in file managment, even does not show up on the computer. the card is fine was able to open on another phone. trying to trouble shoot it for now.
User experienced a malfunctioning SD card slot on their previously loved phone, which is now inaccessible on their new device.

> Finished chain.


In [48]:
result


'User experienced a malfunctioning SD card slot on their previously loved phone, which is now inaccessible on their new device.'